<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Сверточные нейронные сети и </b> <span style="font-weight:bold; color:green">TensorFlow</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_hse@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Загрузка исходных данных</a></li>
        <li><a href="#2">Softmax регрессия</a></li>
        <li><a href="#3">Сверточная нейронная сеть</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#3a">Первый слой</a></li>
                <li><a href="#3b">Второй слой</a></li>
                <li><a href="#3c">Третий и четвертый слои</a></li>
                <li><a href="#3d">Обучение и тестирование</a></li>
                <li><a href="#3e">Dropout-регуляризация</a></li>
            </ol>
        </li>
        <li><a href="#4">Источники</a></li>
    </ol>
</div>

<p>Подключение стилей оформления</p>

In [ ]:
%%html
<link href="css/style.css" rel="stylesheet" type="text/css">

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Загрузка исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Загрузка исходных данных</p>

In [ ]:
mnist = input_data.read_data_sets("data/cnn-mnist", one_hot=True)

<p>Параметры исходных данных</p>

In [ ]:
mnist.train.num_examples

In [ ]:
mnist.validation.num_examples

In [ ]:
mnist.test.num_examples

<p>Доступ к данным</p>

In [ ]:
mnist.train.images[3][150:200]

In [ ]:
mnist.train.labels[3]

In [ ]:
mnist.train.epochs_completed

<p>Отображение исходных данных</p>

In [ ]:
train_digit_img = mnist.train.images[3] * 255
train_digit_img[150:200]

In [ ]:
train_digit_img = train_digit_img.reshape((28, 28))

In [ ]:
plt.figure(figsize=[3,3])
plt.imshow(train_digit_img, cmap="gray")
plt.show()

<p>Создание графа вычислений</p>

In [ ]:
sess = tf.InteractiveSession()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Softmax регрессия</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<div class="msg-block msg-info">
  <p class="msg-text-info">См. предыдущий семинар</p>
</div>

<p>Создание модели</p>

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [ ]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
y = tf.matmul(x,W) + b

In [ ]:
loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y))

<p>Обучение</p>

In [ ]:
learing_rate = 0.5

In [ ]:
train_step = tf.train.GradientDescentOptimizer(learing_rate) \
                            .minimize(loss=loss_cross_entropy, var_list=(W,b))

In [ ]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_true: batch[1]})

<p>Тестирование</p>

In [ ]:
correct_prediction = tf.equal(tf.argmax(input=y, axis=1), tf.argmax(y_true, 1))

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
accuracy.eval(feed_dict={x: mnist.test.images, y_true: mnist.test.labels})

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Сверточная нейронная сеть</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Модель нейронной сети</p>

<img src="images/cnn-mnist/cnn.jpg" width="900px">

<p>Входные и выходные переменные для обучения и тестирования</p>

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

<a name="3a"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Первый слой
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3b">Далее</a>
            </div>
        </div>
    </div>
</div>

<div>
<p>Последовательности операций: <span class="code-font">Image[28x28x1] > <b>CONV1</b> > <b>ReLU</b> > <b>POOL1</b> > Image[14x14x32]</span></p>
<img src="images/cnn-mnist/conv_l1.jpg" width="700px">
</div>

<p>Переопределяем размер входных данных [количество\_изображений, высота, ширина, количество\_каналов]</p>

In [ ]:
x_image = tf.reshape(x, [-1,28,28,1])

<div>
<p>Формирование слоя  COV1:</p>
<ul>
    <li>количество фильтров - 32</li> 
    <li>размер фильтра - 5х5</li>
    <li>количество каналов - 1 (монохромное изображение)</li> 
    <li>шаг смещения - 1</li>
    <li>с сохранением исходной размерности данных (добавление нулей - padding) - да</li>
</ul>
</div>

<p><b>СONV1:</b> <span class="code-font">[-1,28,28,1] > CONV1 [5, 5, 1, 32] > [-1,28,28,32]</span></p>

<p>Начальные значения весов для слоя CONV1</p>

In [ ]:
W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 32], stddev=0.1))

<p>Создание СONV1: [-1,28,28,1] -> conv2d_conv1 [5, 5, 1, 32] -> [-1,28,28,32]</p>

In [ ]:
conv2d_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding="SAME")

<p>Начальное значение смещения для каждого фильтра</p>

In [ ]:
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))

<p><b>ReLU:</b> <span class="code-font">[-1,28,28,32] -> ReLU -> [-1,28,28,32]</span></p>

<p>Cоздание функции активации - ReLU</p>

In [ ]:
h_conv1 = tf.nn.relu(conv2d_conv1 + b_conv1)

<p><b>POOL1:</b> <span class="code-font">[-1,28,28,32] -> POOL1 -> [-1,14,14,32]</span></p>

<div>
<p>Формирование слоя  POOL1:</p>
<ul>
    <li>размер фильтра - 2х2</li>
    <li>количество каналов - 32</li> 
    <li>шаг смещения - 2</li>
    <li>с сохранением исходной размерности данных (добавление нулей - padding) - да</li>
</ul>
</div>

<p>Уменьшение размерности исходных данных до 14x14 - Max 2x2 Pooling</p>

In [ ]:
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

<a name="3b"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Второй слой
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3a">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3c">Далее</a>
            </div>
        </div>
    </div>
</div>

<div>
<p>Последовательность операций: <span class="code-font">Image[14x14x32] > <b>CONV2</b> > <b>ReLU</b> > <b>POOL2</b> > Image[7x7x64]</span></p>
<img src="images/cnn-mnist/conv_l2.jpg" width="700px">
</div>

<div>
<p>Пример с тремя каналами и двумя фильтрами [<a href="http://cs231n.github.io/convolutional-networks/#conv">ссылка</a>]</p>
<img src="images/cnn-mnist/cnn_3ch_2fil.png" width="700px">
</div>

<div>
<p>Формирование слоя  COV2:</p>
<ul>
    <li>количество фильтров - 64</li> 
    <li>размер фильтра - 5х5</li>
    <li>количество каналов - 32 (от каждого фильтра из предыдущего слоя)</li> 
    <li>шаг смещения - 1</li>
    <li>с сохранением исходной размерности данных (добавление нулей - padding) - да</li>
</ul>
</div>

In [ ]:
W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))

In [ ]:
conv2d_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding="SAME")

In [ ]:
h_conv2 = tf.nn.relu(conv2d_conv2 + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

<a name="3c"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            c. Третий и четвертый слои
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3b">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3d">Далее</a>
            </div>
        </div>
    </div>
</div>

<div>
<p>Представление изображений в виде одномерного массива</p>
<img src="images/cnn-mnist/data_repr_fcl.jpg" width="300px">
</div>

<div>
<p><p>Последовательность операций: <span class="code-font">Image[7\*7\*64x1] > <b>FC ReLU</b> > <b>FC Softmax</b> > Probability[10x1] </span></p> 
<img src="images/cnn-mnist/fc_l3_l4.jpg" width="800px">
</div>

<p><b>Третий слой</b></p>

<p>Установка начальных значений весов и смещения</p>

In [ ]:
W_fc1 = tf.Variable(tf.truncated_normal(shape=[7 * 7 * 64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

<p>Преобразование в одномерный массив (для одного исходного изображения)</p>

In [ ]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

<p>Установка функции активации</p>

In [ ]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

<p><b>Четвертый слой</b></p>

<p>Установка начальных значений весов и смещения</p>

In [ ]:
W_fc2 = tf.Variable(tf.truncated_normal(shape=[1024, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

<p>Установка функции активации</p>

In [ ]:
y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

In [ ]:
y_softmax = tf.nn.softmax(y_conv)

<a name="3d"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            d. Обучение и тестирование
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3c">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3e">Далее</a>
            </div>
        </div>
    </div>
</div>

<p><b>Обучение</b></p>

<p>Определение функции потерь - кросс-энтропия</p>

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_softmax), reduction_indices=[1]))

<div class="msg-block msg-warning">
  <p class="msg-text-warn">Для обеспечения вычислительной устойчивости используйте</p>
  <p class="code-block code-font">cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_conv))</p>
  <p>вместо</p>
  <p class="code-block code-font">y_softmax = tf.nn.softmax(y_conv)<br>cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_softmax), reduction_indices=[1]))</p>
</div>

<p>Коэффициент обучения</p>

In [ ]:
learing_rate = 0.001

<p>Установка метрики - доля правильных классификаций/распознаваний (accuracy)</p>

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_softmax,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

<p>Назначение оптимизатора (выполняться будет на batch)</p>

In [ ]:
train_step = tf.train.GradientDescentOptimizer(learing_rate).minimize(loss=cross_entropy)

<div class="msg-block msg-info">
  <p class="msg-text-info">В качестве оптимизатора можно использовать и более продвинутые варианты, например, <span class="code-font">AdamOptimizer</span></p>
  <p class="code-block code-font">
  train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)</p>
</div>

<p><b><i>Запуск вычислений</i></b></p>

<p>Выполнение инициализации переменных</p>

In [ ]:
sess.run(tf.global_variables_initializer())

<p>Итеративный процесс обучения:</p>
<ul>
    <li>всего изображений в обучающем подмножестве - 55000</li>
    <li>размер batch'а - 100</li>
    <li>всего batch'ей - 10000 (возможны повторы)</li>
    <li>количество эпох - 18</li>
</ul>

<div class="msg-block msg-info">
  <p class="msg-text-info">Процесс обучения может занять длительное время (~час). Можно уменьшить количество эпох</p>
</div>

In [ ]:
for i in range(10000): 
    
    batch = mnist.train.next_batch(100)
    
    if i % 50 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_true: batch[1]})
        print("Epoch %i, step %i, training accuracy %f" % (mnist.train.epochs_completed, i, train_accuracy))
    
    train_step.run(feed_dict={x: batch[0], y_true: batch[1]})

<p><b>Тестирование</b></p>

<p>Запуск тестирования получененной модели на данных <span class="code-font">mnist.test</span> и отображение результата</p>

In [ ]:
accuracy.eval(feed_dict={x: mnist.test.images, y_true: mnist.test.labels})

<a name="3e"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            e. Dropout-регуляризация
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3d">Назад</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#4">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Входная переменная для drop-регуляризации</p>

In [ ]:
keep_prob = tf.placeholder(tf.float32)

<p>Добавление dropout-регуляризации в четвертый слой</p>

In [ ]:
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [ ]:
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

<p>Переопределение выражений для обучения</p>

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_conv))
train_step = tf.train.GradientDescentOptimizer(learing_rate).minimize(loss=cross_entropy)

<p>Запуск инициализации всех переменных</p>

In [ ]:
sess.run(tf.global_variables_initializer())

<p>Запуск обучения c вероятностью dropout равной 0.5</p>

In [ ]:
for i in range(10000):
    
    batch = mnist.train.next_batch(100)
    
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_true: batch[1], 
                                                  keep_prob: 1.0})
        
        print("Epoch %i, step %i, training accuracy %f" % (mnist.train.epochs_completed, i, train_accuracy))
        
    train_step.run(feed_dict={x: batch[0], y_true: batch[1], keep_prob: 0.5})

<p>Тестирование</p>

In [ ]:
print "test accuracy %g" % accuracy.eval(feed_dict={x: mnist.test.images, y_true: mnist.test.labels, 
                                                    keep_prob: 1.0})

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<div>
<a href="https://www.tensorflow.org/tutorials/estimators/cnn">Build a Convolutional Neural Network using Estimators</a><br>
<a href="https://www.tensorflow.org/tutorials/deep_cnn">Convolutional Neural Networks</a><br>
<a href="http://cs231n.github.io/convolutional-networks/">Convolutional Neural Networks (CNNs / ConvNets)</a><br>
<a href="http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf">ImageNet Classification with Deep Convolutional Neural Networks (PDF)</a>
</div>